In [14]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
from snorkel.labeling import labeling_function
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier
import random
import nltk
from nltk.corpus import wordnet as wn
from snorkel.augmentation import transformation_function
from snorkel.augmentation import ApplyOnePolicy, PandasTFApplier
from snorkel.slicing import slicing_function
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [15]:
df_test = pd.read_csv('test.csv',index_col=0)
df_train = pd.read_csv('train.csv', index_col=0)

In [ ]:
ABSTAIN = -2
POSITIVE= 1
NEGATIVE = 0
ANNO = -1

In [17]:
def lf_keyword_good(context):
    Positive_word = r"(good|great|well|(be vaccinated)|better)"
    return POSITIVE if re.search(Positive_word, context) else ABSTAIN
def lf_keyword_bad(context):
    Negative_word = r"(cancer|bad|(won't take)|traced|(won't work)|refuse|convinced)"
    return NEGATIVE if re.search(Negative_word, context) else ABSTAIN
def lf_keyword_annoncement(context):
    Anno_word = r"(clinic|volunteer|offically|(find a clinic))"
    return ANNO if re.search(Anno_word, context) else ABSTAIN

In [18]:
def lf_textblob_polarity(x):
    return POSITIVE if TextBlob(x.text).sentiment.polarity > 0.3 else ABSTAIN

In [ ]:
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

lfs = [lf_keyword_good, lf_keyword_bad, lf_keyword_annoncement, lf_textblob_polarity]

applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=200, log_freq=50, seed=123)
df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

df_train = df_train[df_train.label != ABSTAIN]